#Package Installation

In [ ]:
%%capture
!pip install "openai>=1" "langchain>=0.0.331rc2" "unstructured[local-inference]" unstructured_pytesseract langchain_openai langchain_community pypdf pikepdf pdf2image pdfminer.six python-multipart cohere kaleido pillow pydantic lxml pillow matplotlib gradio==3.48.0 chromadb tiktoken

In [ ]:
%%capture
!sudo apt-get install poppler-utils tesseract-ocr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Merging PDFs

In [ ]:
from pypdf import PdfMerger

## Merging both engine manuals and excavator 3015 model together
pdfs = ['/content/drive/MyDrive/pdfFiles/Engine/CAT_C15_Engine_Manual.pdf',
        '/content/drive/MyDrive/pdfFiles/Engine/CAT_C18_Engine_Manual.pdf'
        ]

#'/content/drive/MyDrive/pdfFiles/Excav_3015/CAT_Excavator_Model_3015.pdf'
merger = PdfMerger()

for pdf in pdfs:
    merger.append(pdf)

merger.write("/content/drive/MyDrive/pdfFiles/CAT_Manuals.pdf")
merger.close()

<ipython-input-4-e4eb4279359b>:9: DeprecationWarning: PdfMerger is deprecated and will be removed in pypdf 5.0.0. Use PdfWriter instead.
  merger = PdfMerger()


# Extracting Text, Tables, and Images from the PDF

In [ ]:
from typing import Any
from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf
from langchain.text_splitter import CharacterTextSplitter


# Extract elements from PDF
def extract_pdf_elements(path, fname):
    """
    Extract images, tables, and chunk text from a PDF file.
    path: File path, which is used to dump images (.jpg)
    fname: File name
    """
    return partition_pdf(
        filename=path + fname,
        extract_images_in_pdf=True,
        infer_table_structure=True,
        chunking_strategy="by_title",
        max_characters=4000,
        new_after_n_chars=3800,
        combine_text_under_n_chars=2000,
        image_output_dir_path=path,
    )


# Categorize elements by type
def categorize_elements(raw_pdf_elements):
    """
    Categorize extracted elements from a PDF into tables and texts.
    raw_pdf_elements: List of unstructured.documents.elements
    """
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))
    return texts, tables

# Get elements for Engines and Excavator 3015
raw_pdf_elements_CAT = extract_pdf_elements('/content/drive/MyDrive/pdfFiles/', 'CAT_Manuals.pdf')

# Get text, tables for Engines and Excavators
text_CAT, tables_CAT = categorize_elements(raw_pdf_elements_CAT)

#Enforce a specific token size for texts for Engines and Excavator
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=4000, chunk_overlap=0
)
joined_texts_CAT = " ".join(text_CAT)

texts_4k_token_CAT = text_splitter.split_text(joined_texts_CAT)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#Generating the Text, Table, and Image Summaries

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

os.environ['OPENAI_API_KEY'] = getpass.getpass('Paste API key:')


# Generate summaries of text elements
def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # Text summary chain
    model = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
    elif texts:
        text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

    return text_summaries, table_summaries


# Get text, table summaries
text_summaries_CAT, table_summaries_CAT = generate_text_summaries(
    texts_4k_token_CAT, tables_CAT, summarize_texts=True
)

In [ ]:
import base64
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
import time

def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt):
    """Make image summary"""
    chat = ChatOpenAI(model="gpt-4-turbo", max_tokens=1024)

    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content


def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval."""

    # Apply to images
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image, prompt))

    return img_base64_list, image_summaries

In [ ]:
# Image summaries
img_base64_list_CAT, image_summaries_CAT = generate_img_summaries('/content/figures')

#Creating the Vectorstore and Retrivier

In [ ]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings


def create_multi_vector_retriever(
    vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images
):
    """
    Create retriever that indexes summaries, but returns raw images or texts
    """

    # Initialize the storage layer
    store = InMemoryStore()
    id_key = "doc_id"

    # Create the multi-vector retriever
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Add texts, tables, and images
    # Check that text_summaries is not empty before adding
    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    # Check that table_summaries is not empty before adding
    if table_summaries:
        add_documents(retriever, table_summaries, tables)
    # Check that image_summaries is not empty before adding
    if image_summaries:
        add_documents(retriever, image_summaries, images)

    return retriever

In [ ]:
# The vectorstore to use to index the summaries
vectorstore_CAT = Chroma(
    collection_name="mm_rag_CAT", embedding_function=OpenAIEmbeddings()
)


# Creating the retriever
retriever_multi_vector_img_CAT = create_multi_vector_retriever(
    vectorstore_CAT,
    text_summaries_CAT,
    text_CAT,
    table_summaries_CAT,
    tables_CAT,
    image_summaries_CAT,
    img_base64_list_CAT
)

#Creating the Multi Modal RAG Chain

In [ ]:
import io
import re

from IPython.display import HTML, display
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.schema.messages import SystemMessage
from PIL import Image
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType


def plt_img_base64(img_base64):
    """Disply base64 encoded string as image"""
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # Display the image by rendering the HTML
    display(HTML(image_html))

def looks_like_base64(sb):
    """Check if the string looks like base64"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


def is_image_data(b64data):
    """
    Check if the base64 data is an image by looking at the start of the data
    """
    image_signatures = {
        b"\xFF\xD8\xFF": "jpg",
        b"\x89\x50\x4E\x47\x0D\x0A\x1A\x0A": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False


def resize_base64_image(base64_string, size=(128, 128)):
    """
    Resize an image encoded as a Base64 string
    """
    # Decode the Base64 string
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Resize the image
    resized_img = img.resize(size, Image.LANCZOS)

    # Save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # Encode the resized image to Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def split_image_text_types(docs):
    """
    Split base64-encoded images and texts
    """
    b64_images = []
    texts = []
    for doc in docs:
        # Check if the document is of type Document and extract page_content if so
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc = resize_base64_image(doc, size=(1300, 600))
            b64_images.append(doc)
        else:
            texts.append(doc)
    return {"images": b64_images, "texts": texts}


def img_prompt_func(data_dict):
    """
    Join the context into a single string
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    # Adding the text for analysis
    text_message = {
        "type": "text",
        "text": (
            "You are tasked with providing answers for catepillar.  Only answer questions related to engines and excavators.\n"
            "You will be given a mixed of text, tables, and image(s) usually of charts or graphs.\n"
            "Use this information to provide the answer related to the user question. \n"
            f"User-provided question: {data_dict['question']}\n\n"
            "Text and / or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)
    return [SystemMessage(content=messages)]

## Function for Multi Modal RAG Chain
def multi_modal_rag_chain(retriever):
    """
    Multi-modal RAG chain
    """

    # Multi-modal LLM
    model = ChatOpenAI(temperature=0, model="gpt-4-turbo", max_tokens=1024)

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | model
        | StrOutputParser()
    )

    return chain

In [ ]:
# Create the Multi Modal RAG chain
chain_multimodal_rag_CAT = multi_modal_rag_chain(retriever_multi_vector_img_CAT)

#Creating the Gradio App Interface for the Multi Modal RAG

In [ ]:
import gradio as gr
from PIL import Image
import base64

## Getting response for multi modal rag and image output
def lang_bot(query):
    response_message = chain_multimodal_rag_CAT.invoke(query)
    return response_message

def lang_bot_image(query):
    docs = retriever_multi_vector_img_CAT.get_relevant_documents(query, limit=6)

    ## Converting base64 string to JPG
    for n in range(len(docs)):
      if docs[n].isascii() == True:
        imageData = docs[n]

        # Decode base64 String Data
        decodedData = base64.b64decode((imageData))

        # Write Image from Base64 File to folder
        imgFile = open('/content/drive/MyDrive/pdfFiles/imagefolder/query_image.jpg', 'wb')
        imgFile.write(decodedData)
        imgFile.close()
        break
    return gr.Image("/content/drive/MyDrive/pdfFiles/imagefolder/query_image.jpg")

## Running the gradio app
demo = gr.Blocks()
with demo:
    gr.Markdown("CAT INFO Assistant")
    with gr.Tabs():
        with gr.TabItem("Get Answer"):
            with gr.Row():
                query_txt_input = gr.Textbox(label="Query")
                text_output = gr.Textbox(label="Output")
                image_output = gr.Image(label="Image")

            text_button = gr.Button("Text Run")
            image_button = gr.Button("Image Run")

    text_button.click(lang_bot, inputs=query_txt_input, outputs=text_output)
    image_button.click(lang_bot_image, inputs=query_txt_input, outputs=image_output)

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://bf96bb919df0fb9d7b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://bf96bb919df0fb9d7b.gradio.live
